# Advanced Tool Integration & Function Calling

## Production Agent Capabilities with Enterprise Patterns

**Module Duration:** 15 minutes | **Focus:** Advanced tool patterns with error handling

---

### Learning Objectives

Master production-grade tool integration for enterprise agent systems:

- **Advanced Tool Patterns:** Complex integrations beyond basic functions
- **Error Recovery:** Production retry logic and fallback strategies
- **Tool Composition:** Chaining multiple tools for complex workflows
- **Resource Management:** Rate limiting and connection handling
- **Real Integrations:** File processing, APIs, databases

**What You'll Build:**
- Agent with multiple production-ready tools
- Error handling and retry mechanisms
- Tool orchestration for complex tasks
- Enterprise integration patterns

This covers advanced tool patterns used in real production agent systems.

In [ ]:
# Advanced Tool Integration - Production Patterns
import asyncio
import json
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import logging
from functools import wraps
import sqlite3

# Configure logging for production monitoring
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🛠️ ADVANCED TOOL INTEGRATION")
print("=" * 35)
print(f"Session: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Focus: Production tool patterns with error handling")
print()

@dataclass
class ToolExecutionResult:
    """Production tool execution tracking"""
    tool_name: str
    success: bool
    result: Any
    execution_time: float
    error_message: Optional[str] = None
    retry_count: int = 0

print("✅ Production tool execution framework initialized")
print("   Error tracking, retry logic, performance monitoring")

### Production Tool Decorators

Enterprise tools require error handling, retry logic, and monitoring. These decorators provide:

**Error Recovery Patterns:**
- **Retry Logic:** Exponential backoff for transient failures
- **Circuit Breaker:** Fail-fast when services are down
- **Fallback Strategies:** Alternative approaches when primary tools fail
- **Resource Management:** Rate limiting and connection pooling

In [ ]:
# Production Tool Decorators for Enterprise Reliability

def retry_on_failure(max_retries=3, delay=1.0, backoff=2.0):
    """Enterprise retry decorator with exponential backoff"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            current_delay = delay
            
            for attempt in range(max_retries + 1):
                try:
                    start_time = time.time()
                    result = func(*args, **kwargs)
                    execution_time = time.time() - start_time
                    
                    logger.info(f"{func.__name__} succeeded on attempt {attempt + 1} ({execution_time:.2f}s)")
                    return ToolExecutionResult(
                        tool_name=func.__name__,
                        success=True,
                        result=result,
                        execution_time=execution_time,
                        retry_count=attempt
                    )
                    
                except Exception as e:
                    last_exception = e
                    logger.warning(f"{func.__name__} failed on attempt {attempt + 1}: {e}")
                    
                    if attempt < max_retries:
                        logger.info(f"Retrying in {current_delay:.1f}s...")
                        time.sleep(current_delay)
                        current_delay *= backoff
            
            # All retries failed
            return ToolExecutionResult(
                tool_name=func.__name__,
                success=False,
                result=None,
                execution_time=0.0,
                error_message=str(last_exception),
                retry_count=max_retries
            )
        return wrapper
    return decorator

def rate_limit(calls_per_minute=30):
    """Rate limiting decorator for API tools"""
    call_times = []
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()
            
            # Remove calls older than 1 minute
            call_times[:] = [t for t in call_times if current_time - t < 60]
            
            # Check rate limit
            if len(call_times) >= calls_per_minute:
                sleep_time = 60 - (current_time - call_times[0])
                logger.info(f"Rate limit reached for {func.__name__}. Waiting {sleep_time:.1f}s")
                time.sleep(sleep_time)
            
            call_times.append(current_time)
            return func(*args, **kwargs)
        return wrapper
    return decorator

print("✅ Production tool decorators ready:")
print("   Retry logic with exponential backoff")
print("   Rate limiting for API compliance")
print("   Error tracking and performance monitoring")

### Advanced Tool Implementations

Production agents require sophisticated tools that handle real-world complexity:

**File Processing Tools:**
- **Multi-format Support:** CSV, Excel, JSON, PDF processing
- **Error Handling:** Corrupted files, encoding issues, size limits
- **Performance:** Streaming for large files, memory management

**API Integration Tools:**
- **Authentication:** API keys, OAuth, token refresh
- **Error Recovery:** Network timeouts, rate limits, service errors
- **Data Validation:** Schema validation, type conversion

**Database Tools:**
- **Connection Management:** Pooling, reconnection, transactions
- **Query Safety:** SQL injection prevention, parameter binding
- **Performance:** Query optimization, result streaming

In [ ]:
# Advanced Tool Implementations with Production Patterns

@retry_on_failure(max_retries=2)
@rate_limit(calls_per_minute=20)
def process_data_file(file_path: str, operation: str) -> Dict[str, Any]:
    """
    Advanced file processing with multiple format support and error handling.
    
    Handles CSV, Excel, JSON files with proper error recovery and validation.
    Production features: encoding detection, size limits, memory management.
    """
    import os
    
    # Validate file exists and size
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    file_size = os.path.getsize(file_path)
    if file_size > 50 * 1024 * 1024:  # 50MB limit
        raise ValueError(f"File too large: {file_size / 1024 / 1024:.1f}MB (limit: 50MB)")
    
    file_ext = os.path.splitext(file_path)[1].lower()
    
    try:
        if file_ext == '.csv':
            # CSV processing with encoding detection
            try:
                df = pd.read_csv(file_path, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='latin-1')
            
            if operation == 'analyze':
                return {
                    'rows': len(df),
                    'columns': list(df.columns),
                    'data_types': df.dtypes.to_dict(),
                    'missing_values': df.isnull().sum().to_dict(),
                    'sample_data': df.head(3).to_dict('records')
                }
            elif operation == 'summarize':
                numeric_cols = df.select_dtypes(include=['number']).columns
                return {
                    'total_rows': len(df),
                    'numeric_summary': df[numeric_cols].describe().to_dict() if len(numeric_cols) > 0 else {},
                    'categorical_summary': {col: df[col].value_counts().head().to_dict() 
                                          for col in df.select_dtypes(include=['object']).columns}
                }
                
        elif file_ext in ['.xlsx', '.xls']:
            # Excel processing with sheet handling
            excel_file = pd.ExcelFile(file_path)
            sheets_data = {}
            
            for sheet_name in excel_file.sheet_names[:3]:  # Limit to first 3 sheets
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                sheets_data[sheet_name] = {
                    'rows': len(df),
                    'columns': list(df.columns),
                    'sample': df.head(2).to_dict('records')
                }
            
            return {
                'file_type': 'excel',
                'sheets': sheets_data,
                'total_sheets': len(excel_file.sheet_names)
            }
            
        elif file_ext == '.json':
            # JSON processing with validation
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            return {
                'file_type': 'json',
                'structure': type(data).__name__,
                'size': len(data) if isinstance(data, (list, dict)) else 1,
                'keys': list(data.keys()) if isinstance(data, dict) else None,
                'sample': data[:2] if isinstance(data, list) else data
            }
        
        else:
            raise ValueError(f"Unsupported file type: {file_ext}")
            
    except Exception as e:
        logger.error(f"File processing error: {e}")
        raise

@retry_on_failure(max_retries=3, delay=2.0)
@rate_limit(calls_per_minute=15)
def fetch_api_data(endpoint: str, params: Dict[str, Any] = None) -> Dict[str, Any]:
    """
    Production API integration with authentication and error handling.
    
    Features: timeout handling, response validation, structured error reporting.
    Supports JSON APIs with proper HTTP status code handling.
    """
    if params is None:
        params = {}
    
    # Simulate different API endpoints for demonstration
    if endpoint == 'weather':
        # Mock weather API response
        location = params.get('location', 'Unknown')
        return {
            'location': location,
            'temperature': 22,
            'humidity': 65,
            'conditions': 'Partly Cloudy',
            'timestamp': datetime.now().isoformat(),
            'source': 'mock_weather_api'
        }
    
    elif endpoint == 'stock_data':
        # Mock stock data API
        symbol = params.get('symbol', 'UNKNOWN')
        return {
            'symbol': symbol,
            'price': 145.67,
            'change': '+2.34',
            'change_percent': '+1.63%',
            'volume': 1234567,
            'timestamp': datetime.now().isoformat(),
            'source': 'mock_stock_api'
        }
    
    elif endpoint == 'news':
        # Mock news API
        topic = params.get('topic', 'general')
        return {
            'articles': [
                {
                    'title': f'Latest {topic} news update',
                    'summary': f'Breaking news about {topic} developments.',
                    'published': datetime.now().isoformat(),
                    'source': 'Mock News API'
                },
                {
                    'title': f'{topic.title()} market analysis',
                    'summary': f'Expert analysis on {topic} trends.',
                    'published': (datetime.now() - timedelta(hours=2)).isoformat(),
                    'source': 'Mock News API'
                }
            ],
            'total_results': 2,
            'query': topic
        }
    
    else:
        raise ValueError(f"Unknown API endpoint: {endpoint}")

@retry_on_failure(max_retries=2)
def database_query(query_type: str, table: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
    """
    Production database tool with connection management and query safety.
    
    Features: SQL injection prevention, connection pooling, transaction handling.
    Supports basic CRUD operations with proper error handling.
    """
    if filters is None:
        filters = {}
    
    # Create in-memory database for demonstration
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    try:
        # Create sample tables for demonstration
        if table == 'users':
            cursor.execute('''
                CREATE TABLE users (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    email TEXT,
                    created_date TEXT
                )
            ''')
            
            # Insert sample data
            sample_users = [
                (1, 'Alice Johnson', 'alice@example.com', '2024-01-15'),
                (2, 'Bob Smith', 'bob@example.com', '2024-02-20'),
                (3, 'Carol Davis', 'carol@example.com', '2024-03-10')
            ]
            cursor.executemany('INSERT INTO users VALUES (?, ?, ?, ?)', sample_users)
            
        elif table == 'orders':
            cursor.execute('''
                CREATE TABLE orders (
                    id INTEGER PRIMARY KEY,
                    user_id INTEGER,
                    product TEXT,
                    amount REAL,
                    order_date TEXT
                )
            ''')
            
            sample_orders = [
                (1, 1, 'Laptop', 999.99, '2024-01-20'),
                (2, 2, 'Mouse', 29.99, '2024-02-25'),
                (3, 1, 'Keyboard', 79.99, '2024-03-15')
            ]
            cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?, ?)', sample_orders)
        
        # Execute query based on type
        if query_type == 'select':
            if filters:
                # Build WHERE clause safely
                where_parts = []
                values = []
                for key, value in filters.items():
                    where_parts.append(f"{key} = ?")
                    values.append(value)
                
                where_clause = " AND ".join(where_parts)
                query = f"SELECT * FROM {table} WHERE {where_clause}"
                cursor.execute(query, values)
            else:
                cursor.execute(f"SELECT * FROM {table}")
            
            columns = [description[0] for description in cursor.description]
            rows = cursor.fetchall()
            
            return {
                'query_type': 'select',
                'table': table,
                'columns': columns,
                'rows': [dict(zip(columns, row)) for row in rows],
                'count': len(rows)
            }
            
        elif query_type == 'count':
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            
            return {
                'query_type': 'count',
                'table': table,
                'total_records': count
            }
            
        else:
            raise ValueError(f"Unsupported query type: {query_type}")
            
    finally:
        conn.close()

print("\n🔧 Advanced Tools Ready:")
print("   File processing: CSV, Excel, JSON with error handling")
print("   API integration: Weather, stock, news with retry logic")
print("   Database tools: Safe queries with connection management")
print("   Production patterns: Rate limiting, monitoring, validation")

### Agent Integration with Advanced Tools

Now we integrate these production tools with ADK agents. The agent will:

**Tool Selection Logic:**
- **Capability Matching:** Choose appropriate tools based on request type
- **Error Recovery:** Fallback strategies when tools fail
- **Resource Awareness:** Respect rate limits and connection pools
- **Result Validation:** Verify tool outputs before responding

In [43]:
# ✅ Agent Integration with Advanced Tools (Google ADK 1.3.0)

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types
import os
from collections import defaultdict

# Track tool usage across runs (optional)
tool_usage_counter = defaultdict(int)

# ✅ Tool definitions

def analyze_file(file_path: str) -> str:
    tool_usage_counter["analyze_file"] += 1
    print(f"🔧 Tool Called: analyze_file({file_path})")

    result = process_data_file(file_path, "analyze")

    if hasattr(result, 'success') and result.success:
        data = result.result
        return f"File Analysis: {data['rows']} rows, {len(data['columns'])} columns. Columns: {', '.join(data['columns'][:5])}{'...' if len(data['columns']) > 5 else ''}"
    return f"File analysis failed: {getattr(result, 'error_message', 'Unknown error')}"

def get_weather_data(location: str) -> str:
    tool_usage_counter["get_weather_data"] += 1
    print(f"🔧 Tool Called: get_weather_data({location})")

    result = fetch_api_data("weather", {"location": location})

    if hasattr(result, 'success') and result.success:
        data = result.result
    else:
        data = result  # fallback
    return f"Weather in {data['location']}: {data['temperature']}°C, {data['conditions']}, Humidity: {data['humidity']}%"

def get_stock_info(symbol: str) -> str:
    tool_usage_counter["get_stock_info"] += 1
    print(f"🔧 Tool Called: get_stock_info({symbol})")

    result = fetch_api_data("stock_data", {"symbol": symbol})

    if hasattr(result, 'success') and result.success:
        data = result.result
    else:
        data = result
    return f"Stock {data['symbol']}: ${data['price']} ({data['change']}, {data['change_percent']}), Volume: {data['volume']:,}"

def search_database(table: str, user_filter: str) -> str:
    tool_usage_counter["search_database"] += 1
    print(f"🔧 Tool Called: search_database(table='{table}', user_filter='{user_filter}')")

    if not isinstance(user_filter, str):
        print(f"[⚠️ Warning] Unexpected type for user_filter: {type(user_filter)}. Forcing string.")
        user_filter = str(user_filter)

    if user_filter.strip():
        return f"Filtered results from {table} where {user_filter}"
    return f"All records from {table}"

# ✅ Async setup function

async def setup_advanced_agent():
    """Initialize agent with advanced tool capabilities"""

    model = LiteLlm(model="ollama_chat/llama3.2:latest")

    # Optional Gemini switch:
    # model = LiteLlm(
    #     model="gemini/gemini-1.5-flash",
    #     api_key=os.getenv("GOOGLE_API_KEY")
    # )

    agent = Agent(
        name="AdvancedToolAgent",
        model=model,
        instruction="""
You are a production-ready AI agent with advanced tool capabilities.

You can:
- Analyze structured data files (CSV, Excel, JSON)
- Fetch real-time information like weather and stock prices
- Search internal databases with or without filters
- Use multiple tools together for complex tasks

When using the 'search_database' tool, always pass 'user_filter' as plain text like 'age > 30' or an empty string. Do not pass JSON or objects.

When using tools:
1. Select the most appropriate tool
2. Handle errors gracefully
3. Always explain which tools you used and why
4. Return clear and helpful responses
""",
        tools=[
            analyze_file,
            get_weather_data,
            get_stock_info,
            search_database
        ]
    )

    session_service = InMemorySessionService()
    runner = Runner(agent=agent, app_name="advanced_tools", session_service=session_service)

    await session_service.create_session(
        app_name="advanced_tools",
        user_id="system",
        session_id="main"
    )

    return agent, runner, session_service

# ✅ Initialize the agent
agent, runner, session_service = await setup_advanced_agent()

print("\n✅ Advanced Tool Agent Ready:")
print(" - File analysis: CSV, Excel, JSON")
print(" - API data: Weather and stock prices")
print(" - Database queries with filters")
print(" - Multi-tool orchestration and reasoning")



✅ Advanced Tool Agent Ready:
 - File analysis: CSV, Excel, JSON
 - API data: Weather and stock prices
 - Database queries with filters
 - Multi-tool orchestration and reasoning


### Production Tool Demonstration

Testing the agent with complex requests that require multiple tools and error handling:

**Test Scenarios:**
- **Multi-tool Requests:** Tasks requiring multiple integrated tools
- **Error Recovery:** Handling tool failures gracefully
- **Tool Chaining:** Using output from one tool as input to another
- **Resource Management:** Respecting rate limits and timeouts

In [45]:
# ✅ Production Tool Demonstration with Google ADK Agent (Any LLM via LiteLLM)

from collections import defaultdict
import asyncio

# Counter to track tool usage across test runs
tool_usage_counter = defaultdict(int)

# Example for logging tool invocations:
# def get_weather_data(city):
#     tool_usage_counter["get_weather_data"] += 1
#     print(f"[TOOL USED] get_weather_data({city})")
#     ...

async def test_advanced_tools():
    """Run a suite of tests on an ADK-based agent with tool integration."""

    test_requests = [
        "Get the current weather in San Francisco and stock price for AAPL",
        "Search the orders table where amount > 500",
        "Get weather for New York, then get stock info for TSLA and GOOGL",
        "Find all orders in the database and get current weather in Chicago"
    ]

    print("\n🧪 ADVANCED TOOL INTEGRATION TESTING")
    print("=" * 42)

    results = []

    for i, request in enumerate(test_requests, 1):
        print(f"\n📋 Test {i}: {request}")
        print("-" * 50)

        message = types.Content(
            role="user",
            parts=[types.Part(text=request)]
        )

        response_text = ""

        async for event in runner.run_async(
            user_id="system",
            session_id="main",
            new_message=message
        ):
            if event.is_final_response():
                response_text = event.content.parts[0].text
                break

        print(f"🤖 Agent Response:\n{response_text}")
        results.append({'request': request, 'response': response_text})

        await asyncio.sleep(1)  # Avoid overwhelming APIs

    return results


# Run the tool demonstration
demo_results = await test_advanced_tools()

print("\n📊 TOOL INTEGRATION ANALYSIS")
print("=" * 35)

print("🛠️ Tool Invocation Notes:")
print("   - Tools are triggered implicitly based on model output.")
print("   - No need to define structured `function_call` schemas.")
print("   - Use print logs inside each tool to confirm execution.")
print("   - Use counters to track frequency and validate behavior.")

# Summary of tool usage
if tool_usage_counter:
    print("\n🔢 TOOL USAGE SUMMARY")
    for tool, count in tool_usage_counter.items():
        print(f"   - {tool}: {count} call(s)")

print("\n✅ ADVANCED TOOL INTEGRATION COMPLETE")
print("   - Agent handled tool-based tasks using the configured LLM.")
print("   - Continue testing edge prompts to validate robustness.")


09:33:44 - LiteLLM:INFO: utils.py:3101 - 
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat



🧪 ADVANCED TOOL INTEGRATION TESTING

📋 Test 1: Get the current weather in San Francisco and stock price for AAPL
--------------------------------------------------


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
09:33:57 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:__main__:fetch_api_data succeeded on attempt 1 (0.00s)
INFO:__main__:fetch_api_data succeeded on attempt 1 (0.00s)
09:33:59 - LiteLLM:INFO: utils.py:3101 - 
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat


🔧 Tool Called: get_weather_data(San Francisco)
🔧 Tool Called: get_stock_info(AAPL)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
09:34:01 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
09:34:14 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


...


📋 Test 4: Find all orders in the database and get current weather in Chicago
--------------------------------------------------


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
09:35:38 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:__main__:fetch_api_data succeeded on attempt 1 (0.00s)
09:35:40 - LiteLLM:INFO: utils.py:3101 - 
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.2:latest; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
09:35:40 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Reques

🔧 Tool Called: search_database(table='orders', user_filter='')
🔧 Tool Called: get_weather_data(Chicago)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
09:35:55 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
09:35:57 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:LiteLLM:selected model name for cost calculation: ollama_chat/llama3.2:latest
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


🤖 Agent Response:
I used the 'search_database' tool to search for all records in the 'orders' table. This resulted in a full return of all data from the 'orders' table.

Then, I used the 'get_weather_data' tool to retrieve the current weather in Chicago. The result shows that the current weather in Chicago is 22°C with a partly cloudy sky and a humidity level of 65%.

📊 TOOL INTEGRATION ANALYSIS
🛠️ Tool Invocation Notes:
   - Tools are triggered implicitly based on model output.
   - No need to define structured `function_call` schemas.
   - Use print logs inside each tool to confirm execution.
   - Use counters to track frequency and validate behavior.

🔢 TOOL USAGE SUMMARY
   - get_weather_data: 3 call(s)
   - get_stock_info: 3 call(s)
   - search_database: 2 call(s)

✅ ADVANCED TOOL INTEGRATION COMPLETE
   - Agent handled tool-based tasks using the configured LLM.
   - Continue testing edge prompts to validate robustness.
